In [1]:
import folium
import pandas as pd
from haversine import haversine, Unit

# My stored variables from another jupyters

In [2]:
%store -r
%store

Stored variables and their in-db values:
facilities              -> ['airport', 'airport', 'airport', 'airport', 'airp
lat_airport             -> ['-6.081689834590001', -5.20707988739, -5.82678985
lon_airport             -> ['145.391998291', 145.789001465, 144.2960052490234
office_lat              -> [34.090368, 33.022176, 40.7464969, 53.344104, 34.0
office_long             -> [-118.393064, -117.081406, -74.0094471, -6.267494,
office_name             -> ['Geni', 'Slacker', 'Joost', 'Babelgum', 'Mahalo',
star_lat                -> [42.51, 25.42, 25.39, 24.48, 24.51, 24.4, 24.4, 24
star_long               -> [1.53, 55.47, 55.47, 54.38, 54.54, 54.49, 54.49, 5
starbucks               -> ['Meritxell, 96', 'Ajman Drive Thru', 'Dana Mall',


-----------
# FUNCTIONS

# - Measuring proximity

## closest_and_ubi function using haversine formula
- takes 2 lists (a,b) of geospatial coordinates (as long as required both, lengh-independent)
- the output is:
    - for all elements on a, the closest point to all elements of b
    - the coordinates of the closest point of b
- how it works_
    name_list_for_distances, name_list_for_coordinates = closest_and_ubi (list1, list2)
    
it is not allowed to storage functions in one jupyter-notebook and call them from another. You get a RaiseError output. So I define my functions below

In [3]:
def closest_and_ubi(offices_ubi,airports_ubi):
    '''    Esta función necesita correrse como a,b=closest_and_ubi(elem1,elem2)'''
    ubi_to_office=[]
    closest_airport = []
    for o in offices_ubi:
        distancia=[]
        airports=[]
        for a in airports_ubi:
            #if haversine(o,a)<200:
            distancia.append(haversine(o,a))
            airports.append(a)
            
        m=min(distancia)
        air=airports[distancia.index(m)]
        ubi_to_office.append(round(m,1))  #the result is given gith uncertainty of +- 0.1km
        closest_airport.append(air)

    return ubi_to_office, closest_airport

# - Creating coord tuples from lists of coordinates
- I got lists of longitudes and latitudes for all places i'm working with in different jupyter-notebooks. I'll turn it into tuples in order to use the haversine library

In [4]:
def coord_tuple(list_lat,list_long):    
    '''- input: 2 lists. list1: latitudes, list2:longitudes
    - output: list of tuples with coordinates
    - how to: coordenadas=tuple(list_lat,list_ong)'''
    
    airports_ubi=[]
    for lat,long in zip(list_lat,list_long):
        ubication=(float(lat),float(long))
        airports_ubi.append(ubication)
    return airports_ubi

# - Geolocating with Folium

In [76]:
def map_adding(lista,nombre,colores):
    '''
    lista== column of the final dataset
    nombre== kind of facility
    colores == colour for this kind of facility
    '''
    for i in lista:
        folium.CircleMarker(
            location=[i[0],i[1]],
            radius=40,
            popup=nombre,
            color=colores,
            fill=True,
            fill_color=colores
        ).add_to(world_map)

# Creating coord tuples with my ubications and getting the list of names(1st approach)

In [6]:
airports_ubi=coord_tuple(lat_airport,lon_airport)

In [7]:
starbucks_ubi=coord_tuple(star_lat,star_long)

In [8]:
offices_ubi=coord_tuple(office_lat,office_long)

# Getting the closest distance from offices
- to all airports in my dataset (the most important in the world)
- I tried the scipy module for this purpose, but all i found it was uncompatible with gps coordinates, so, i downloaded the haversine library and worked with it

In [9]:
# using the function defined above
distance_airport_km, ubi_airport = closest_and_ubi(offices_ubi,airports_ubi)

In [73]:
distance_starbucks_km, ubi_starbucks=closest_and_ubi(offices_ubi,starbucks_ubi)

# Creating my dataframe with all the information:

In [74]:
data=pd.DataFrame({"offices":offices_ubi, 
                   "distance_mobility_point(km)":distance_airport_km,
                   "distance_starbucks(km)":distance_starbucks_km,
                   "coords_airport":ubi_airport,
                   "coords_starbucks":ubi_starbucks})


In [12]:
#data.head(10

# Weight function to find the perfect spot
- I created a simple way to rank the preferred location
    - starbucks have great weight
    - it is not neccessary to have an airport very close, so airports weight reduce them in the function

In [13]:
data['Reverse_rank'] =(data["distance_mobility_point(km)"]*0.02+data["distance_starbucks(km)"]*100)

In [14]:
data=data.sort_values(by=["Reverse_rank"])

In [15]:
final_data=data.reset_index(drop=True)

In [16]:
final_data.head(5)

,offices,distance_mobility_point(km),distance_starbucks(km),coords_airport,coords_starbucks,Reverse_rank
0,"(47.6103008, -122.3399782)",0.3,0.0,"(47.611969, -122.33708200000001)","(47.61, -122.34)",0.006
1,"(45.500277, -73.560084)",0.5,0.0,"(45.499722, -73.56611099999999)","(45.5, -73.56)",0.010
2,"(40.739867, -73.990024)",1.2,0.0,"(40.7505, -73.9935)","(40.74, -73.99)",0.024
3,"(45.5309644, -122.68038)",0.4,0.1,"(45.528999999999996, -122.6768)","(45.53, -122.68)",10.008
4,"(37.779624, -122.391236)",0.5,0.1,"(37.776388399999995, -122.39444440000001)","(37.78, -122.39)",10.010


# Folium

In [17]:
# Creating name labels:


In [38]:
world_map=folium.Map(location=[final_data["offices"][0][0],final_data["offices"][0][1]])

In [19]:
#world_map

In [68]:
map_adding(final_data["offices"][0:5], "offices","red")

In [69]:
map_adding(final_data["coords_airport"][0:5],"mobility_point","blue")

In [70]:
map_adding(final_data["coords_starbucks"][0:5],"starbucks","green")

In [71]:
world_map                 

In [30]:
    folium.Marker([i[0],i[1]],
                 radius=4,
                 icon=folium.Icon(icon='cloud',color='blue'),
                 popup=facilities[i]).add_to(world_map)
    

TypeError: list indices must be integers or slices, not tuple

In [54]:
# Airports

In [55]:
for i in range(len(facilities)):
    folium.Marker([lat_airport[i],lon_airport[i]],
                 radius=2,
                 icon=folium.Icon(icon='cloud',color='blue'),
                 popup=facilities[i]).add_to(world_map)

# World map

In [39]:
world_map